# Behind the pipeline (PyTorch)

In [2]:
! pip install transforms datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and p

In [12]:
from transformers import pipeline, AutoTokenizer, AutoModel, AutoModelForSequenceClassification

In [13]:
classifier = pipeline(task='sentiment-analysis')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [14]:
classifier(["Im bored","she is excited about being a girl",])

[{'label': 'NEGATIVE', 'score': 0.9997822642326355},
 {'label': 'POSITIVE', 'score': 0.9997207522392273}]

In [15]:
checkpoint='distilbert-base-uncased-finetuned-sst-2-english'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
raw_inputs = ["Im bored","she is excited about being a girl",]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [37]:
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors='pt')
print (type(inputs.input_ids),inputs.input_ids.shape, inputs)

<class 'torch.Tensor'> torch.Size([2, 9]) {'input_ids': tensor([[  101, 10047, 11471,   102,     0,     0,     0,     0,     0],
        [  101,  2016,  2003,  7568,  2055,  2108,  1037,  2611,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [38]:
model = AutoModel.from_pretrained(checkpoint)
output = model(**inputs)
print (type(output.last_hidden_state), output.last_hidden_state.shape)

<class 'torch.Tensor'> torch.Size([2, 9, 768])


In [45]:
classifier_2 = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs_2 = classifier_2(**inputs)
print(outputs_2.logits.shape)
print(outputs_2.logits)

torch.Size([2, 2])
tensor([[ 4.6653, -3.7668],
        [-3.9465,  4.2367]], grad_fn=<AddmmBackward0>)


In [50]:
import torch
prediction = torch.nn.functional.softmax(outputs_2.logits, dim=-1)
print(prediction)
classifier_2.config.id2label

tensor([[9.9978e-01, 2.1770e-04],
        [2.7924e-04, 9.9972e-01]], grad_fn=<SoftmaxBackward0>)


{0: 'NEGATIVE', 1: 'POSITIVE'}